| Aspect                       | **Vector Space Model (VSM)**                                             | **Extended Boolean Model (EBM)**                                            |
| :--------------------------- | :----------------------------------------------------------------------- | :-------------------------------------------------------------------------- |
| **What it measures**         | *Similarity* — how close a document is to the query in a geometric sense | *Satisfaction* — how well a document fulfills a Boolean condition (AND, OR) |
| **Mathematical idea**        | Based on **angles** between vectors (cosine)                             | Based on **p-norms** that generalize AND/OR logic                           |
| **Query type**               | Free weighted terms (like “AI, data”)                                    | Boolean-style queries (“AI AND data”, “AI OR data”)                         |
| **Interpretation of result** | “Document is 0.8 similar to query”                                       | “Document satisfies 80% of the Boolean condition”                           |
| **Behavior control**         | Fixed (no parameters)                                                    | Adjustable (via parameter ( p ))                                            |
| **Origin**                   | Linear algebra                                                           | Fuzzy logic + geometry                                                      |


## Vector Space Model (VSM) for Information Retrieval

In this notebook, we implement the Vector Space Model using TF–IDF weighted document vectors and compute similarity between queries and documents using three measures:

- **Inner Product Similarity**
- **Cosine Similarity**
- **Jaccard Similarity**

We test the model on the following queries:

1. `q1`: large language models for information retrieval and ranking  
2. `q2`: LLM for information retrieval and Ranking  
3. `q3`: query Reformulation in information retrieval  
4. `q4`: ranking Documents  
5. `q5`: Optimizing recommendation systems with LLMs by leveraging item metadata

6. **Load the Document–Term and Inverted Index files**  
   - Use the files generated in Lab 1 to access the TF–IDF weights of terms in each document.

7. **Preprocess each query**  
   - Tokenize the query text.  
   - Remove stop words.  
   - Apply stemming (e.g., Porter Stemmer).

8. **For each similarity measure (Inner Product, Cosine, Jaccard):**  
   - Compute the similarity score between the query and every document.  
   - Use a **binary weighting scheme** for queries:  
     - Weight = 1 if the term appears in the query.  
     - Weight = 0 if the term does not appear.  
   - Rank the documents in **descending order of similarity**.  
   - Display the ranked documents for each query.


In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict
import re

# -------------------------------
# 1️⃣ Load TF–IDF Weighted Inverted Index
# -------------------------------
inverted_path = "results/inverted_index_weighted.txt"

data = pd.read_csv(
    inverted_path, 
    sep="\t", 
    header=None, 
    names=["term", "doc", "freq", "tfidf"]
)

# Build dictionary: {doc: {term: tfidf}}
doc_dict = defaultdict(dict)
for _, row in data.iterrows():
    term = str(row["term"]).lower()
    doc = str(row["doc"]).replace(".txt", "")
    doc_dict[doc][term] = float(row["tfidf"])

# -------------------------------
# 2️⃣ Preprocessing Function for Queries
# -------------------------------
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess(text):
    tokens = re.findall(r"\b\w+\b", text.lower())
    filtered = [stemmer.stem(t) for t in tokens if t not in stop_words]
    return filtered

# -------------------------------
# 3️⃣ Queries
# -------------------------------
queries = {
    "q1": "large language models for information retrieval and ranking",
    "q2": "LLM for information retrieval and Ranking",
    "q3": "query Reformulation in information retrieval",
    "q4": "ranking Documents",
    "q5": "Optimizing recommendation systems with LLMs by leveraging item metadata"
}

# -------------------------------
# 4️⃣ Vector Space Model Scoring Functions
# -------------------------------
def inner_product(query_terms, doc_terms):
    score = 0.0
    for term in query_terms:
        if term in doc_terms:
            score += doc_terms[term] * 1  # query weight = 1
    return score

def cosine_similarity(query_terms, doc_terms):
    numerator = inner_product(query_terms, doc_terms)
    doc_norm = np.sqrt(sum(np.square(list(doc_terms.values()))))
    query_norm = np.sqrt(len(query_terms))  # binary weights
    if doc_norm == 0 or query_norm == 0:
        return 0
    return numerator / (doc_norm * query_norm)

def jaccard_similarity(query_terms, doc_terms):
    intersection = sum([doc_terms[t] for t in query_terms if t in doc_terms])
    union = len(query_terms) + sum(doc_terms.values()) - intersection
    return intersection / union if union != 0 else 0

# -------------------------------
# 5️⃣ Compute and Rank Similarities
# -------------------------------
def rank_documents(query_text):
    query_terms = preprocess(query_text)
    scores = []

    for doc, terms in doc_dict.items():
        inner = inner_product(query_terms, terms)
        cosine = cosine_similarity(query_terms, terms)
        jaccard = jaccard_similarity(query_terms, terms)
        scores.append((doc, inner, cosine, jaccard))

    df = pd.DataFrame(scores, columns=["Document", "InnerProduct", "Cosine", "Jaccard"])
    df = df.sort_values(by="Cosine", ascending=False)
    return df.reset_index(drop=True)

# -------------------------------
# 6️⃣ Run for All Queries
# -------------------------------
for q_id, q_text in queries.items():
    print(f"\n🔹 Results for {q_id}: {q_text}")
    results = rank_documents(q_text)
    display(results.head(10))



🔹 Results for q1: large language models for information retrieval and ranking


,Document,InnerProduct,Cosine,Jaccard
0,D4,0.556701,0.197726,0.038190
1,D2,0.852919,0.185778,0.039867
2,D3,0.426031,0.129475,0.025982
3,D1,0.541854,0.120376,0.026686
4,D5,0.500000,0.104187,0.021837
5,D6,0.200258,0.056124,0.012284



🔹 Results for q2: LLM for information retrieval and Ranking


,Document,InnerProduct,Cosine,Jaccard
0,D2,0.944483,0.251957,0.048931
1,D4,0.537050,0.233616,0.042634
2,D5,0.541393,0.138167,0.025959
3,D1,0.361236,0.098286,0.019542
4,D6,0.263757,0.090533,0.018524
5,D3,0.220954,0.082242,0.015132



🔹 Results for q3: query Reformulation in information retrieval


,Document,InnerProduct,Cosine,Jaccard
0,D4,1.088431,0.473465,0.090361
1,D1,0.840824,0.228774,0.046698
2,D5,0.265293,0.067704,0.012554
3,D3,0.049743,0.018515,0.003367
4,D2,0.066323,0.017693,0.003286
5,D6,0.049743,0.017074,0.003442



🔹 Results for q4: ranking Documents


,Document,InnerProduct,Cosine,Jaccard
0,D2,0.761100,0.287137,0.043527
1,D1,0.215836,0.083050,0.012978
2,D5,0.079520,0.028700,0.004117
3,D6,0.000000,0.000000,0.000000
4,D4,0.000000,0.000000,0.000000
5,D3,0.000000,0.000000,0.000000



🔹 Results for q5: Optimizing recommendation systems with LLMs by leveraging item metadata


,Document,InnerProduct,Cosine,Jaccard
0,D3,0.848528,0.238747,0.049988
1,D6,0.515043,0.133637,0.030319
2,D4,0.314068,0.103274,0.019853
3,D5,0.421943,0.081400,0.017599
4,D2,0.342423,0.069052,0.014950
5,D1,0.095424,0.019626,0.004387


## 🧩 Explanation

### 🔹 Inverted Index
- Loaded from your **Lab 1 output** (contains: *term*, *document*, *TF–IDF*).  
- Allows efficient retrieval of documents containing each term.

---

### 🔹 Document Dictionary (`doc_dict`)
- Maps every **document → its TF–IDF terms**.  
- Example:  
  `{ "doc1": {"cat": 0.23, "mat": 0.18, ...}, "doc2": {...} }`

---

### 🔹 Query Preprocessing
- Removes **stopwords**.  
- Applies **stemming** to reduce words to their root form.  
- Uses **binary weighting** for queries → each query term = **1**.

---

### 🔹 Similarity Measures

| Measure | Description |
|:---------|:-------------|
| **Inner Product** | Basic overlap strength between query and document vectors. |
| **Cosine Similarity** | Normalized measure of similarity (most commonly used). |
| **Jaccard Similarity** | Balances overlap with document length. |

---

### 🔹 Ranking
- Documents are **sorted by cosine similarity** by default.  
- You can change the sorting column to use **another similarity metric**.


The Vector Space Model measures how similar documents are to a query.

The Extended Boolean Model measures how much a document satisfies a Boolean-style query — but with graded results.